# BioE 101 Lab 1 - Analyzing Frequency Spectrums#

## Objectives: ##

- Learn how to program with Python (or at least become more familiar with Python)

- Use Python to see your voice with your computer's microphone in the time and frequency domain

- Use an Arduino with a function generator to sample waveforms

- Observe aliasing and quantization error in sampled data

- Estimate noise and SNR from sampled data

- (Bonus) Build an algorithm with Python to distinguish you and your partner’s voice or build a digital filter for noisy data

There will be two weeks for this lab. Each group will have to turn in a lab report with the answers to all the questions below. There are 7 questions with 2 bonus questions. For the bonus questions, you must turn in your code as well to show proof of an attempt.

#### Before starting the lab:  
Download everything in the Lab 1 Folder on bCourses > Files > Lab 1 to the same folder as your lab1 notebook file

#### Lab Report Submission:
Write down the answers in the notebook or a Word Doc. Everyone in the group each needs to turn in the lab report. Make sure to include all your names and student IDs

## Part 1. Displaying signals in time and frequency domain with Python

We're going to generate waveform inputs into the Arduino and view it in the time and frequency domains. The Arduino has a 10-bit ADC built into it. That puts a limit on how small our input can be. ADCs take in an analog voltage and outputs a "code" that goes from $0$ to $2^{B}$ where $B$ is the number of bits. In the Arduino's case, that means that the ADCs will read in a voltage that goes from 0V to 3.3V and map that to $0$ to $1023$.

**Question 1**: If our ADC is 10 bits, what is the smallest voltage difference we can observe between any two points?

### Learn to plot a function and its spectrum in Python ###
Run the cell below to import some necessary libraries. Numpy is used for scientific-computing (i.e., lots of linear algebra and DSP). Matplotlib is a plotting library very similar to how matplot makes its plots.

In [ ]:
import numpy as np # import numpy to create functions
import matplotlib.pyplot as plt # import the plotting library
from scipy.fftpack import fft # fft function from scipy
# set matplotlib to plot the graphs inside the notebook
%matplotlib inline

Run the cell below to generate a sine wave and plot its Fourier Transform. Feel free to play around with the values! A little run-down of what's going on:
- np.arange(n) creates an a n-size array going from 0 to n-1
- np.sin(x) creates an array that evaluates sin at every value in the array of x
- plt.plot(x,y) plots y with respect to x

In [ ]:
def plot_ft_sinewave(sample_freq, freq):
    Fs = sample_freq;  # sampling rate
    Ts = 1.0/Fs; # sampling interval
    t = np.arange(0,Ts*500,Ts) # time vector - sample duration of 1 sec

    ff = freq;   # frequency of the signal
    y = np.sin(2*np.pi*ff*t)

    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[range(n//2)] # one side frequency range

    Y = fft(y)/n # fft computing and normalization
    Y = Y[range(n//2)]

    fig, ax = plt.subplots(2, 1,figsize=(20,10))
    ax[0].plot(t,y)
    ax[0].set_xlabel('Time')
    ax[0].set_ylabel('Amplitude')
    ax[1].plot(frq,abs(Y),'r') # plotting the spectrum
    ax[1].set_xlabel('Freq (Hz)')
    ax[1].set_ylabel('|Y(freq)|')
    plt.show()

plot_ft_sinewave(1000, 100)

**Question 2**: Let's play around with some values. Plot a 1 kHz sine wave sampled at 3k samples/second.

In [ ]:
plot_ft_sinewave(*,*) # replace * with the appropriate sampling frequency and sine wave frequency

**Question 3**: Plot a 400 Hz sine wave sampled at 1k samples/second and plot a 600 Hz sine wave sampled at 1k samples/second. What do you notice, and why is this happening? What would you change about how these signals were acquired to prevent this phenomenon.

In [ ]:
# Plot 400 Hz
plot_ft_sinewave(*,*) # replace * with the appropriate sampling frequency and sine wave frequency
# Plot 600 Hz
plot_ft_sinewave(*,*) # replace * with the appropriate sampling frequency and sine wave frequency

## Part 2. Observe audio signals in time and frequency domains using your microphone.

Now that we've successfully used our time domain and frequency domain analysis tools on python generated signals, let's actually use real world signals. For audio signals in particular, frequency analysis tools are pretty useful.

Make sure someone in your group has a computer with a microphone built-in and PyAudio installed.

If running the cell below gives an error related to matplotlib or tk, then in your terminal run these commands:  
$\texttt{conda update matplotlib}$  
$\texttt{conda update ipython}$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyaudio
import struct
from scipy.fftpack import fft
import sys
import time
%matplotlib tk

### Audio Spectrum Analyzer
Below is a class object that uses your microphone to plot your voice in time and in the frequency domain. Run the cell to define the class.

In [31]:
class AudioStream(object):
    def __init__(self):

        # stream constants
        self.CHUNK = 1024 * 20 # play around with this value for Question 4, keep as multiple of 1024
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = 1
        self.RATE = 44100
        self.pause = False

        # stream object
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(
            format=self.FORMAT,
            channels=self.CHANNELS,
            rate=self.RATE,
            input=True,
            output=True,
            frames_per_buffer=self.CHUNK,
        )
        self.init_plots()
        self.start_plot()

    def init_plots(self):

        # x variables for plotting
        x = np.arange(0, 2 * self.CHUNK, 2)
        xf = np.linspace(0, self.RATE, self.CHUNK)

        # create matplotlib figure and axes
        self.fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))
        self.fig.canvas.mpl_connect('button_press_event', self.onClick)

        # create a line object with random data
        self.line, = ax1.plot(x, np.random.rand(self.CHUNK), '-', lw=2)

        # create line for spectrum
        self.line_fft, = ax2.plot(
            xf, np.random.rand(self.CHUNK), '-', lw=2)

        # format waveform axes
        ax1.set_title('AUDIO WAVEFORM')
        ax1.set_xlabel('samples')
        ax1.set_ylabel('volume')
        ax1.set_ylim(-50, 300)
        ax1.set_xlim(0, 2 * self.CHUNK)
        plt.setp(
            ax1, yticks=[0, 128, 255],
            xticks=[0, self.CHUNK, 2 * self.CHUNK],
        )
        plt.setp(ax2, yticks=[-100, -50, 0],)

        # format spectrum axes
        ax2.set_xlim(20, 2000)

        # show axes
        thismanager = plt.get_current_fig_manager()
        # thismanager.window.setGeometry(5, 120, 1910, 1070)
        plt.show(block=False)

    def start_plot(self):

        print('stream started')
        frame_count = 0
        start_time = time.time()

        try:
            while not self.pause:
                data = self.stream.read(self.CHUNK, exception_on_overflow=False)
                data_int = struct.unpack(str(2 * self.CHUNK) + 'B', data)
                data_np = np.array(data_int, dtype='b')[::2] + 128

                self.line.set_ydata(data_np)

                # compute FFT and update line
                yf = np.abs(fft(data_int))
                yf_norm = 20*np.log10(yf/np.amax(yf))
                self.line_fft.set_ydata(yf_norm[0:self.CHUNK])
                    
                # update figure canvas
                self.fig.canvas.draw()
                self.fig.canvas.flush_events()
                frame_count += 1
        except Exception as e:
            print(e)
            self.fr = frame_count / (time.time() - start_time)
            print('average frame rate = {:.0f} FPS'.format(self.fr))
            self.exit_app()

    def exit_app(self):
        print('stream closed')
        self.p.close(self.stream)

    def onClick(self, event):
        self.pause = True

Now run the cell below to instantiate the audio spectrum analyzer. This should pop open a window where you can view your voice in time and frequency. Try humming or whistling low, medium and high pitch notes and see what happens to the frequency spectrum!

In [32]:
AudioStream()

stream started
invalid command name "."
average frame rate = 2 FPS
stream closed


**Question 4**:  Try increasing self.CHUNK to smaller or larger multiples of 1024. Besides the spectrum analyzer being slower, what else do you notice, and why do you think this is happening? Don't worry if you aren't too familiar with the Discrete Fourier Transform, just provide a little intuition. (Hint: with a smaller number of data points read for the DFT, are you able to distinguish between two frequencies that are close together?)

## Part 3. Observe a sine wave in time and frequency domains using the Arduino

Before hooking up the function generator to the Arduino, always check the waveform with an oscilloscope to make sure the output is what you want.

Using the same procedure as the end of lab 0: 
- Connect the function generator to the A0 and GND pins of the Arduino and upload the adc_sampling.ino sketch to the Arduino board. 
- Set the sine wave to be 50 Hz with 400mVpp and 1.5 V DC offset. 
    - **Any waveform is okay as long as none of the waveform dips below 0V or goes above 3.3V**
- Turn on the function generator and view the serial plotter using Tools->Serial Plotter. You should see the sine wave.

The Python code below is a little more in-depth tutorial on how to use "fft" and "ifft" fucntion form "scipy.fftpack" to do fourier transfrom and inverse fourier transform. In the given Arduino file, you already send the file to Serial after each run. Here, we import the data from Serial and do Fourier Analysis for it. After you run the Arduino file, please run the code below.

**Things to check before running:**
- Make sure the interval in the Arduino sketch is 1000 - later you can change this if you'd like
- Make sure your waveform is okay and your equipment works
- Make sure you know the COM (or /dev/tty if on Linux/Mac) port of your Arduino (explained in the next block)

Instead of the Serial Plotter, we will now be using the Python code below to plot and analyze our signals. The code below imports the necessary Python libraries:

In [ ]:
# allows plots to be plotted right below the cell when run
%matplotlib inline 
import serial # the library for reading from serial com ports
import numpy as np # naming convention for numpy library
import matplotlib.pyplot as plt # naming convention for matplotlib
from scipy.fftpack import fft, ifft # import discrete fourier transform and its inverse

Before proceeding, find the serial port for your arduino. On Windows, check Device Manager and go under Ports and find the Arduino COM Port. On Macs, go to the terminal and type ls /dev/tty.\* and look for the port corresponding to the Arduino.

Also make sure to close the Serial monitor or plotter since that is accessing the serial port and only one program can access the serial port at a time.

Now, run the block below to define a sampling function that samples your serial data for 3 seconds (you can edit this to to sample for any arbitrary amount of time).

In [ ]:
def sample_arduino():
    # preamble to set up serial communications
    device = "COM9" # com port of Arduino <- CHANGE THIS TO THE ONE YOU FOUND
    baud = 115200
    ser = serial.Serial(device, baud, timeout=10)

    # read in every line of serial code as floats and ignore corrupt data
    def sample(size):
        i = 0
        data = []
        while i < size:
            data.append(try_float(ser.readline().decode("utf-8", "ignore").strip('\n').strip('\r')))
            i += 1
        return data
    def try_float(s):
        try:
            return float(s)
        except:
            return 0
    raw_data = sample(3000) # <- Adjust this line to read in more/less data
    ser.close()
    return raw_data




Run the following block of code to sample and plot your waveform:

In [ ]:
signal = sample_arduino()
plt.plot(signal[100:200]) # change the indices to zoom in/out in time

To take a fourier transform of the above signal and analyze its frequency spectrum, run the code below:

In [ ]:
dft = fft(signal) # calculate fourier transform
fs = 1000 # sampling rate
N = len(dft) # length of discrete fourier transform
freqs = [i*fs/N for i in range(N)] # convert from dft frequencies to Hz
plt.plot(freqs[2:1000], np.abs(dft[2:1000])) # change the indices to zoom in/out in frequency

Now you're able to sample a chunk of data from the Arduino's analog inputs at a time and process it. This is cool, but we can do better! Similar to the Audio Spectrum Analyzer, let's view the spectrum of the Arduino's analog input in real time. Run the cell below to run a script that will generate a spectrum analyzer for the Arduino. Leave it running as you do Question 5 and 6. 

#### Before running the cell, open up the python file in a text editor of your choice and change the "device = " line to include the correct COM or /dev/ port of your Arduino.

In [10]:
%matplotlib tk
%run serial_spectrumQT.py COM11

COM11


SerialException: could not open port 'COM11': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

**Question 5**: Similar to Part 1, we'll try increasing frequencies until some weird things happen. Now increase the frequency of the function generator (suggestion: step 50 Hz at a time starting at 50 Hz). At what point does the frequency not match your input? 

**Question 6**: What property of LTI systems does this violate? In general, why is sampling not an LTI operator?

## Part 4: Voice Recognition ##

For your convenience if you want to just do this section, run the cell below to import the necessary libraries.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pyaudio
import struct
from scipy.fftpack import fft
import sys
import time
%matplotlib tk

In this section, you will develop an algorithm to distinguish between your and your partner's voices using Python and your computer's microphone.

While speaking into the microphone, make sure to be loud and clear. Try things like saying a specific word or humming/whistling a note!

Run the cell below to define a function that will be used to record your voice and plot and return your voice data in time and frequency.

In [18]:
def record_voice():
    CHUNK = 1024 * 10
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 4000
    pause = False

    p = pyaudio.PyAudio()
    stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RATE,
        input=True,
        output=True,
        frames_per_buffer=CHUNK,
    )
    # x variables for plotting
    x = np.arange(0, 2 * CHUNK, 2)
    xf = np.linspace(0, RATE//2, CHUNK)

    # create matplotlib figure and axes
    fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

    # create a line object with random data
    line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

    # create line for spectrum
    line_fft, = ax2.plot(
        xf, np.random.rand(CHUNK), '-', lw=2)

    # format waveform axes
    ax1.set_title('AUDIO WAVEFORM')
    ax1.set_xlabel('samples')
    ax1.set_ylabel('volume')
    ax1.set_ylim(-50, 300)
    ax1.set_xlim(0, 2 * CHUNK)
    plt.setp(
        ax1, yticks=[0, 128, 255],
        xticks=[0, CHUNK, 2 * CHUNK],
    )
    plt.setp(ax2, yticks=[-100, -50, 0],)

    # format spectrum axes
    ax2.set_xlim(20, 2000)
    ax2.set_ylim(-100, 0) # adjust this line if your signal is clipped off in the fourier domain
    data = stream.read(CHUNK, exception_on_overflow=False)
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    data_np = np.array(data_int, dtype='b')[::2] + 128
    line.set_ydata(data_np)

    # compute FFT and update line
    yf = np.abs(fft(data_int))
    yf_norm = 20*np.log10(yf/np.amax(yf))
    line_fft.set_ydata(yf_norm[0:CHUNK])
    data_ft = np.abs(yf[0:CHUNK])
    
    return np.array(data_int), data_ft[1:RATE//2]/CHUNK

In [19]:
voice1 = record_voice()
print("Partner 1")
# SAVE PARTNER 1 DATA #
partner_1_data, partner_1_ft = voice1

[  0.         -40.77342336 -50.42421932 ..., -49.65504085 -50.42421932
 -40.77342336]
Partner 1


In [ ]:
voice2 = record_voice()
print("Partner 2")
# SAVE PARTNER 2 DATA #
partner_2_data, partner_2_ft = voice2

In [ ]:
voice3 = record_voice()
print("Partner 3")
# SAVE PARTNER 3 DATA #
partner_3_data, partner_3_ft = voice3

** Question 7**: How clean are the voice samples? What kinds of noises/interferences may be present?

## Bonus Questions
- attempting one will give you partial bonus points, attempting both will give more partial bonus points
- successfully implementing one or both will net you half or full bonus points respectively - successful implementation will be defined as a working code that runs and does what it's meant to do, but not necessarily be accurate in something like detecting whose voice it is.
- attach your code to your lab report if you've attempted the bonus for some credit and get checked off by a GSI if you think you've successfully implemented the bonus for full credit

**Bonus Question 1**: Now that you've stored all of your group members' voices, create a program that takes in a voice sample and outputs the person whose voice matches it the most. (Hint: there's an algorithm called cross-correlation that calculates the similarity between two signals. You can use np.correlate(partner_1_ft, partner_2_ft) to compare two voices)

**Bonus Question 2**: You might have noticed that your signal was a little noisy. Create a program that digitally filters out that noise (you get to choose your cutoff frequencies). (Hint: try using a digital filter from the scipy library like the one here https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.signal.lfilter.html)